<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>

# 第四章：从头实现一个GPT模型用于生成文本

In [1]:
from importlib.metadata import version

print("matplotlib version:", version("matplotlib"))
print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

matplotlib version: 3.7.5
torch version: 2.5.1+cu121
tiktoken version: 0.8.0


- 本章实现一个GPT架构的模型，下一章则是聚焦在如何训练这个模型

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch04_compressed/01.webp" width="500px">
</p>

## 4.1 编写LLM架构
- 第一章中讨论了如GPT、Llama结构的模型，此类模型以序列的方式生成文本，是基于原始transformer架构中的decoder部分构建
- 因此，LLMs往往被称为仅编码器的LLMs
- 与传统的深度学习模型比较，LLMs更大，主要是参数量大，而不是实现代码多
- 可以在LLMs的架构中看到很多模块重复

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch04_compressed/02.webp" width="400px">
</p>

- 在之前的章节中，使用对输出、输出使用较小的嵌入层维度，便于进行解释用途
- 在本章中，将按照小型的GPT2模型的架构设置参数
- 将专门编写最小的 GPT-2 模型（1.24 亿参数）的架构，正如 Radford 等人在 [Language Models are Unsupervised Multitask Learners](https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf) 中所概述的那样（请注意，初始报告中列出的参数量是 1.17 亿，但在模型权重仓库中后来被更正）
- 第六章会展示如何加载本章节实现的LLMs模型的预训练参数如何加载，该实现将兼容 3.45 亿、7.62 亿和 15.42 亿参数的模型规模

- 1.24 亿参数的GPT2模型的详细配置参数如下

In [2]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # 词表大小
    "context_length": 1024, # 支持的最长上下文长度，1K
    "emb_dim": 768,         # 嵌入层维度
    "n_heads": 12,          # 注意力头数
    "n_layers": 12,         # 层数
    "drop_rate": 0.1,       # Dropout率
    "qkv_bias": False       # 是否使用偏置
}

- 以下使用简短的变量名以避免之后出现过长的代码行
- `"vocab_size"` 表示词汇量大小为50,257个词，这由第2章讨论的BPE分词器支持
- `"context_length"` 代表模型的最大输入标记数量，这由第2章介绍的位置嵌入实现
- `"emb_dim"` 是标记输入的嵌入大小，将每个输入标记转换为768维向量
- `"n_heads"` 是第3章实现的多头注意力机制中的注意力头数量
- `"n_layers"` 是模型内部transformer块的数量，将在接下来的章节中实现
- `"drop_rate"` 是第3章讨论的dropout机制的强度；0.1意味着在训练期间丢弃10%的隐藏单元以减轻过拟合
- `"qkv_bias"` 决定多头注意力机制（来自第3章）中的Linear层在计算查询(Q)、键(K)和值(V)张量时是否应包含偏置向量；将禁用此选项，这是现代LLM的标准做法；但是，在第5章将OpenAI的预训练GPT-2权重加载时，将重新讨论这一点

<p align="center">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch04_compressed/03.webp" width="400px">
</p>

In [3]:
from typing import Dict

import torch
import torch.nn as nn

In [ ]:
class DummyTransformerLayer(nn.Module):  # 实现的是一个身份变换层，输入什么就输出什么；可以起到占位符的作用
    def __init__(self, cfg: Dict) -> None:
        super().__init__()
        # 模拟一个transformer块的结构
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return x
    

class DummyLayerNorm(nn.Module):
    def __init__(self, normalized_shape, eps=1e-5):
        super().__init__()
        # 模拟一个layer norm层

    def forward(self, x):
        return x


class DummyGPTModel(nn.Module):
    def __init__(self, cfg: Dict) -> None:
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.dropout = nn.Dropout(cfg["drop_rate"])
        
        # 先使用一个占位符类实现整体架构
        self.trf_blocks = nn.Sequential(*[DummyTransformerLayer(cfg) for _ in range(cfg["n_layers"])])

